In [13]:
import time
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import re
from scipy.spatial.distance import cdist
from datetime import date
from numpy import asarray
import seaborn as sns
import math

pd.options.mode.chained_assignment = None  # default='warn'

#Import Datasets
season_df = pd.read_csv('/Users/yushunli/Documents/Data_Science/nfl_player_similarity/data/2013_to_2022_data.csv')
draft_df = pd.read_csv('/Users/yushunli/Documents/Data_Science/nfl_player_similarity/data/1994_to_2022_draftclass.csv')
#Create a Position Rank columns by Season
season_df['Pos_Rank'] = season_df.groupby(['Pos', 'Season'])['Fantasy_Points'].rank(ascending = False, method = 'min')

In [37]:
target = 'Deebo Samuel'

In [44]:
#Run Similarity Analysis
#st.dataframe(find_peers(season_df = season_df, target = target))

#Run Find Peers Function
df = season_df.copy(deep=True)
target_df = df.loc[df.Player == target]
position = target_df.Pos.iloc[0]  
min_age = target_df.Age.min()
max_age = target_df.Age.max()
peer_df = df.loc[(df.Age >= min_age) & (df.Age <= max_age) & (df.Pos == position)] 

# #Run the Fantasy Points abs. difference function

peer_df = peer_df.drop_duplicates(subset = ['Player', 'Age'], keep='first')
peer_pivot = peer_df.pivot(index = 'Player', columns = 'Age', values = 'Fantasy_Points').dropna(axis=0)

In [52]:
reference_row = peer_pivot.loc[peer_pivot.index == target].iloc[0]
peer_fantasy = round(abs(peer_pivot.sub(reference_row) / reference_row),2)
peer_fantasy.columns = 'Age_' + peer_fantasy.columns.astype(int).astype(str)
peer_fantasy['Avg'] = round(peer_fantasy.mean(axis = 1),2)
peer_fantasy = peer_fantasy.loc[peer_fantasy.index != target]
peer_fantasy.sort_values(by = 'Avg', ascending = True, inplace=True)
peer_fantasy.head(10)
# compare = peer_fantasy.sort_values(by = 'Avg', ascending = True)
# fantasy_points = compare.loc[compare.index != target]
# fantasy_points = fantasy_points.sort_index()
# fantasy_points.sort_values(by = 'Avg', ascending = True, inplace=True)
# fantasy_points
# #return fantasy_points

Age,Age_23,Age_24,Age_25,Age_26,Avg
Player,,,,,
Tavon Austin,0.37,0.00,0.43,0.20,0.25
Kendrick Bourne,0.40,0.27,0.65,0.07,0.35
Sterling Shepard,0.03,0.77,0.50,0.20,0.38
Will Fuller,0.37,0.41,0.62,0.17,0.39
Albert Wilson,0.51,0.03,0.67,0.43,0.41
...,...,...,...,...,...
Randall Cobb,0.42,2.81,0.45,0.16,0.96
Michael Thomas,0.36,2.22,0.12,1.16,0.97
Davante Adams,0.50,2.27,0.36,0.97,1.02


In [20]:
season_df.loc[season_df.Player == 'Lamar Jackson']

,Player,Pos,Season,Fantasy_Points,Age,G,Cmp,Pass_Att,Pass_Yds,Pass_TD,...,Rush_Yds_Scaled,Rush_TD_Scaled,Tgt,Rec,Rec_Yds,Rec_TD,Rec_Scaled,Rec_Yds_Scaled,Rec_TD_Scaled,Pos_Rank
2,Lamar Jackson,QB,2019,402.0,22.0,15.0,265.0,401.0,3127.0,36.0,...,0.784794,0.437500,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
48,Lamar Jackson,QB,2020,319.0,23.0,15.0,242.0,376.0,2757.0,26.0,...,0.498774,0.411765,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0
287,Lamar Jackson,QB,2021,230.0,24.0,12.0,246.0,382.0,2882.0,16.0,...,0.427632,0.111111,0.0,0.0,0.0,0.0,0.0,0.0,0.0,16.0
291,Lamar Jackson,QB,2022,228.0,25.0,12.0,203.0,326.0,2242.0,17.0,...,0.467026,0.176471,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13.0
934,Lamar Jackson,QB,2018,142.0,21.0,16.0,99.0,170.0,1201.0,6.0,...,0.490345,0.294118,0.0,0.0,0.0,0.0,0.0,0.0,0.0,29.0
